### The goal of this notebook is to demonstrate the following:
- programmatic query of calibration files for any of the active instruments
- using the results, chunk all of the darks into sets of darks covering a ~30 day period
    - these chunks of darks will then be combined to reject cosmic rays
    - For the time being I will **ignore** anneal boundaries. All cosmic rays affecting only a few pixels are rejected and so any hot pixels that are cured will be ignored across the anneal boundary will be removed
#### Author: Nate Miles

In [2]:
from astroquery.mast import Observations
from collections import defaultdict
from astropy.time import Time
import pandas as pd

- Originally, I was querying for every single dark every taken, then I was filtering that table by extension and date of observations.

- Instead, I should be filtering by date beforehand to get a much smaller table and then filter that table by extension to get only the RAW and SPT dataproducts

- In order to acheive this in a generalized manner without having to hard code dates, I will query for all darks, sort by date and then extract the first and last values to get the start and stop times

In [20]:
#Observations.enable_s3_hst_dataset()

In [6]:
obsTable = Observations.query_criteria(project='HST',
                                      instrument_name = 'STIS/CCD',
                                      obstype='cal',
                                      target_name = 'DARK')
df = obsTable.to_pandas()

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35532 entries, 0 to 35531
Data columns (total 31 columns):
dataproduct_type         35532 non-null object
calib_level              35532 non-null float64
obs_collection           35532 non-null object
obs_id                   35532 non-null object
target_name              35532 non-null object
s_ra                     35532 non-null float64
s_dec                    35532 non-null float64
t_min                    35532 non-null float64
t_max                    35532 non-null float64
t_exptime                35532 non-null float64
wavelength_region        35532 non-null object
filters                  35532 non-null object
em_min                   113 non-null float64
em_max                   113 non-null float64
target_classification    35532 non-null object
obs_title                35510 non-null object
t_obs_release            35532 non-null float64
instrument_name          35532 non-null object
proposal_pi              35532 non-null 

In [23]:
start = Time(min(obsTable['t_min']), format='mjd')
stop = Time(max(obsTable['t_min']), format='mjd')
print(start.iso, stop.iso)
# very roundabout way of generating a list of MJD dates separated by a month
dates = pd.date_range(start=start.iso, end=stop.iso, freq='1MS')

dates = [Time(date.date().isoformat(),format='iso') for date in dates]

1997-02-28 01:30:41.360 2018-06-23 00:49:25.853


In [24]:
dates

[<Time object: scale='utc' format='iso' value=1997-03-01 00:00:00.000>,
 <Time object: scale='utc' format='iso' value=1997-04-01 00:00:00.000>,
 <Time object: scale='utc' format='iso' value=1997-05-01 00:00:00.000>,
 <Time object: scale='utc' format='iso' value=1997-06-01 00:00:00.000>,
 <Time object: scale='utc' format='iso' value=1997-07-01 00:00:00.000>,
 <Time object: scale='utc' format='iso' value=1997-08-01 00:00:00.000>,
 <Time object: scale='utc' format='iso' value=1997-09-01 00:00:00.000>,
 <Time object: scale='utc' format='iso' value=1997-10-01 00:00:00.000>,
 <Time object: scale='utc' format='iso' value=1997-11-01 00:00:00.000>,
 <Time object: scale='utc' format='iso' value=1997-12-01 00:00:00.000>,
 <Time object: scale='utc' format='iso' value=1998-01-01 00:00:00.000>,
 <Time object: scale='utc' format='iso' value=1998-02-01 00:00:00.000>,
 <Time object: scale='utc' format='iso' value=1998-03-01 00:00:00.000>,
 <Time object: scale='utc' format='iso' value=1998-04-01 00:00:0

In [25]:
date_ranges = zip(dates[::2], dates[1::2])

In [26]:
dates = next(date_ranges)

In [27]:
obsTable = Observations.query_criteria(project='HST',
                                      instrument_name='STIS/CCD',
                                      obstype ='cal',
                                      target_name='DARK',
                                      t_min=[dates[0].mjd, dates[1].mjd],
                                      t_exptime=[800, 2000])

In [28]:
products = Observations.get_product_list(obsTable)

In [29]:
products

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid
str10,str3,str8,str9,str62,str1,str35,str9,str28,str3,str1,str7,str22,str4,str18,int64,str10
2003480136,HST,image,o3sj01p8q,DADS TRL file - Processing log,S,mast:HST/product/o3sj01p8q_trl.fits,AUXILIARY,--,TRL,--,CALSTIS,--,7085,o3sj01p8q_trl.fits,23040,2003480136
2003480139,HST,image,o3sj02pfq,DADS RAW file - Raw exposure COS/NICMOS/STIS/WFC3/ACS,S,mast:HST/product/o3sj02pfq_raw.fits,SCIENCE,--,RAW,--,CALSTIS,--,7085,o3sj02pfq_raw.fits,2255040,2003480139
2003480147,HST,image,o3sj02poq,DADS SPT file - Engineering telemetry ACS/WFC3/NICMOS/COS/STIS,S,mast:HST/product/o3sj02poq_spt.fits,AUXILIARY,--,SPT,--,CALSTIS,--,7085,o3sj02poq_spt.fits,69120,2003480147
2003480150,HST,image,o3sj03pvq,DADS JIF file,S,mast:HST/product/o3sj03pvj_jif.fits,AUXILIARY,--,JIF,--,CALSTIS,--,7085,o3sj03pvj_jif.fits,17280,2003480150
2003480169,HST,image,o3sj04qtq,DADS JIF file,S,mast:HST/product/o3sj04qtj_jif.fits,AUXILIARY,--,JIF,--,CALSTIS,--,7085,o3sj04qtj_jif.fits,17280,2003480169
2003480169,HST,image,o3sj04qtq,DADS TRL file - Processing log,S,mast:HST/product/o3sj04qtq_trl.fits,AUXILIARY,--,TRL,--,CALSTIS,--,7085,o3sj04qtq_trl.fits,23040,2003480169
2003480169,HST,spectrum,o3sj04qtq,DADS FLT file - Calibrated exposure ACS/WFC3/STIS/COS,S,mast:HST/product/o3sj04qtq_flt.fits,SCIENCE,Minimum Recommended Products,FLT,--,CALSTIS,2.22 (27-June-2006),7085,o3sj04qtq_flt.fits,10532160,2003480169
2003480187,HST,image,o3sj06rhq,DADS RAW file - Raw exposure COS/NICMOS/STIS/WFC3/ACS,S,mast:HST/product/o3sj06rhq_raw.fits,SCIENCE,--,RAW,--,CALSTIS,--,7085,o3sj06rhq_raw.fits,2255040,2003480187
2003480189,HST,image,o3sj06rkq,DADS JIT file,S,mast:HST/product/o3sj06rkj_jit.fits,AUXILIARY,--,JIT,--,CALSTIS,--,7085,o3sj06rkj_jit.fits,11520,2003480189


In [30]:
products.columns

<TableColumns names=('obsID','obs_collection','dataproduct_type','obs_id','description','type','dataURI','productType','productGroupDescription','productSubGroupDescription','productDocumentationURL','project','prvversion','proposal_id','productFilename','size','parent_obsid')>

In [31]:
filtered_products = Observations.filter_products(products,
                                                 mrp_only=False,
                                                 productSubGroupDescription=['FLT','SPT'])

In [32]:
filtered_products


obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid
str10,str3,str8,str9,str62,str1,str35,str9,str28,str3,str1,str7,str22,str4,str18,int64,str10
2003480147,HST,image,o3sj02poq,DADS SPT file - Engineering telemetry ACS/WFC3/NICMOS/COS/STIS,S,mast:HST/product/o3sj02poq_spt.fits,AUXILIARY,--,SPT,--,CALSTIS,--,7085,o3sj02poq_spt.fits,69120,2003480147
2003480169,HST,spectrum,o3sj04qtq,DADS FLT file - Calibrated exposure ACS/WFC3/STIS/COS,S,mast:HST/product/o3sj04qtq_flt.fits,SCIENCE,Minimum Recommended Products,FLT,--,CALSTIS,2.22 (27-June-2006),7085,o3sj04qtq_flt.fits,10532160,2003480169
2003480189,HST,image,o3sj06rkq,DADS SPT file - Engineering telemetry ACS/WFC3/NICMOS/COS/STIS,S,mast:HST/product/o3sj06rkq_spt.fits,AUXILIARY,--,SPT,--,CALSTIS,--,7085,o3sj06rkq_spt.fits,69120,2003480189
2003480202,HST,spectrum,o3sj07bmq,DADS FLT file - Calibrated exposure ACS/WFC3/STIS/COS,S,mast:HST/product/o3sj07bmq_flt.fits,SCIENCE,Minimum Recommended Products,FLT,--,CALSTIS,2.23 (06-October-2006),7085,o3sj07bmq_flt.fits,10532160,2003480202
2003480207,HST,image,o3sj08bsq,DADS SPT file - Engineering telemetry ACS/WFC3/NICMOS/COS/STIS,S,mast:HST/product/o3sj08bsq_spt.fits,AUXILIARY,--,SPT,--,CALSTIS,--,7085,o3sj08bsq_spt.fits,69120,2003480207
2003480143,HST,image,o3sj02pjq,DADS SPT file - Engineering telemetry ACS/WFC3/NICMOS/COS/STIS,S,mast:HST/product/o3sj02pjq_spt.fits,AUXILIARY,--,SPT,--,CALSTIS,--,7085,o3sj02pjq_spt.fits,69120,2003480143
2003480152,HST,image,o3sj03pxq,DADS SPT file - Engineering telemetry ACS/WFC3/NICMOS/COS/STIS,S,mast:HST/product/o3sj03pxq_spt.fits,AUXILIARY,--,SPT,--,CALSTIS,--,7085,o3sj03pxq_spt.fits,69120,2003480152
2003480154,HST,image,o3sj03pzq,DADS SPT file - Engineering telemetry ACS/WFC3/NICMOS/COS/STIS,S,mast:HST/product/o3sj03pzq_spt.fits,AUXILIARY,--,SPT,--,CALSTIS,--,7085,o3sj03pzq_spt.fits,69120,2003480154
2003480174,HST,image,o3sj05qzq,DADS SPT file - Engineering telemetry ACS/WFC3/NICMOS/COS/STIS,S,mast:HST/product/o3sj05qzq_spt.fits,AUXILIARY,--,SPT,--,CALSTIS,--,7085,o3sj05qzq_spt.fits,69120,2003480174


In [33]:
filtered_products.columns

<TableColumns names=('obsID','obs_collection','dataproduct_type','obs_id','description','type','dataURI','productType','productGroupDescription','productSubGroupDescription','productDocumentationURL','project','prvversion','proposal_id','productFilename','size','parent_obsid')>

In [34]:
download_list = filtered_products['obsID'].tolist()

In [35]:
download_list

['2003480147',
 '2003480169',
 '2003480189',
 '2003480202',
 '2003480207',
 '2003480143',
 '2003480152',
 '2003480154',
 '2003480174',
 '2003480209',
 '2003480211',
 '2003480218',
 '2003480222',
 '2003480130',
 '2003480132',
 '2003480134',
 '2003480145',
 '2003480172',
 '2003480183',
 '2003480185',
 '2003480191',
 '2003480194',
 '2003480205',
 '2003480209',
 '2003480139',
 '2003480152',
 '2003480161',
 '2003480163',
 '2003480213',
 '2003480216',
 '2003480128',
 '2003480139',
 '2003480141',
 '2003480150',
 '2003480150',
 '2003480165',
 '2003480167',
 '2003480185',
 '2003480202',
 '2003480134',
 '2003480136',
 '2003480145',
 '2003480156',
 '2003480161',
 '2003480165',
 '2003480187',
 '2003480196',
 '2003480207',
 '2003480128',
 '2003480136',
 '2003480156',
 '2003480174',
 '2003480176',
 '2003480180',
 '2003480130',
 '2003480183',
 '2003480187',
 '2003480220',
 '2003480227',
 '2003480227',
 '2003480229',
 '2003480233',
 '2003480231',
 '2003480235',
 '2003480337',
 '2003480343',
 '20034803

In [38]:
manifest = Observations.download_products(download_list[:10],
                                          download_dir='./../crrejtab/STIS/',
                                          mrp_only=False,
                                         productSubGroupDescription=['FLT','SPT'])